In [1]:
import pytorch_lightning as pl
from omegaconf import OmegaConf, DictConfig
import nemo.collections.asr as nemo_asr
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager
import datetime
from pathlib import Path
import torch
from sbb_project import consts
import copy
from pytorch_lightning.loggers import WandbLogger  
import wandb
from ruamel.yaml import YAML
import functools
import copy

[NeMo W 2023-01-02 18:39:40 optimizers:67] Could not import distributed_fused_adam optimizer from Apex
[NeMo W 2023-01-02 18:39:41 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/torch/jit/annotations.py:309: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
      warnings.warn("TorchScript will treat type annotations of Tensor "
    


In [2]:
sweep_config = {
  "method": "random",   # Random search
  "metric": {           # We want to minimize `val_loss`
      "name": "val_wer",
      "goal": "minimize"
  },
  "parameters": {
        "lr": {
            # log uniform distribution between exp(min) and exp(max)
            "distribution": "log_uniform",
            "min": -9.21,   # exp(-9.21) = 1e-4
            "max": -2.30    # exp(-4.61) = 1e-2
        },
      "freq_masks": {
            "distribution": "int_uniform",
            "min": 0,  
            "max": 7     
        },
      "time_masks": {
            "distribution": "int_uniform",
            "min": 0,  
            "max": 15     
        },
        "freq_width": {
            "distribution": "uniform",
            "min": 25,  
            "max": 30     
        },
          "time_width": {
                "distribution": "uniform",
                "min": 0.01,  
                "max": 0.07     
            }      
    }
}

In [3]:
train_manifest = [
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "none"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "neg10"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "0"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "10"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "20"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "30"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "40"))),
    ]
val_manifest = consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("val", "random"))
test_manifest = consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("test", "random"))

In [4]:
def sweep_iteration():
    
    # set up W&B logger
    wandb.init()    # required to have access to `wandb.config`
    wandb_logger = WandbLogger(log_model='all')  # log final model
        
    trainer = pl.Trainer(max_epochs=10, logger=wandb_logger, devices = [3], accelerator="gpu")
    
    # setup model - note how we refer to sweep parameters with wandb.config
    model = nemo_asr.models.ASRModel.from_pretrained(model_name='stt_en_squeezeformer_ctc_large_ls',
                                                    map_location=torch.device("cuda:3"))

    model.set_trainer(trainer)
    
    model.cfg.train_ds.is_tarred = False
    
    model.cfg.train_ds.manifest_filepath = ','.join(train_manifest)
    model.cfg.validation_ds.manifest_filepath = str(val_manifest)
    model.cfg.test_ds.manifest_filepath = str(test_manifest)
    
    model.cfg.train_ds.max_duration = 45
    model.cfg.train_ds.batch_size = 4
    model.cfg.validation_ds.batch_size = 4
    model.cfg.test_ds.batch_size = 4
    
    model.cfg.optim.lr = wandb.config.lr
    model.cfg.spec_augment.freq_width = wandb.config.freq_width
    model.cfg.spec_augment.freq_masks = wandb.config.freq_masks
    model.cfg.spec_augment.time_width = wandb.config.time_width
    model.cfg.spec_augment.time_masks = wandb.config.time_masks

    model.setup_training_data(model.cfg.train_ds)
    model.setup_validation_data(model.cfg.validation_ds)
    model.setup_test_data(model.cfg.test_ds)
    model.setup_optimization(model.cfg.optim)

    # train
    trainer.fit(model)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="ASR-Squeezeformer")
wandb.agent(sweep_id, function=sweep_iteration)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: 82i3208e
Sweep URL: https://wandb.ai/unisg-ds-nlp/ASR-Squeezeformer/sweeps/82i3208e


wandb: Agent Starting Run: hzx2m2gj with config:
wandb: 	freq_masks: 4
wandb: 	freq_width: 25.34341064429223
wandb: 	lr: 0.00883928463040186
wandb: 	time_masks: 6
wandb: 	time_width: 0.06198047510138381
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marccgrau (unisg-ds-nlp). Use `wandb login --relogin` to force relogin


[NeMo W 2023-01-02 18:39:49 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
      rank_zero_warn(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2023-01-02 18:39:50 cloud:56] Found existing object /home/user/.cache/torch/NeMo/NeMo_1.14.0/stt_en_squeezeformer_ctc_large_ls/09fe0f2bd920387f15b95317ab64e2ef/stt_en_squeezeformer_ctc_large_ls.nemo.
[NeMo I 2023-01-02 18:39:50 cloud:62] Re-using file from: /home/user/.cache/torch/NeMo/NeMo_1.14.0/stt_en_squeezeformer_ctc_large_ls/09fe0f2bd920387f15b95317ab64e2ef/stt_en_squeezeformer_ctc_large_ls.nemo
[NeMo I 2023-01-02 18:39:50 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2023-01-02 18:39:51 mixins:170] Tokenizer SentencePieceTokenizer initialized with 128 tokens


...

[NeMo I 2023-01-02 23:13:01 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:13:01 wer_bpe:261] reference:rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:13:01 wer_bpe:262] predicted:rangierllmhrt vt antworten
[NeMo I 2023-01-02 23:13:01 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:13:01 wer_bpe:261] reference:rangierfahrt vom gleis sieben via zulu acht ins gleis null null antworten
[NeMo I 2023-01-02 23:13:01 wer_bpe:262] predicted:rangierllmhrt vt antworten
[NeMo I 2023-01-02 23:13:01 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:13:01 wer_bpe:261] reference:spracheingabe
[NeMo I 2023-01-02 23:13:01 wer_bpe:262] predicted:rangierllmhrt vt antworten
[NeMo I 2023-01-02 23:13:01 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:13:01 wer_bpe:261] reference:rangierfahrt von quebec f ⁇ nf acht ins gleis oscar sieben drei antworten
[NeMo I 2023-01-02 23:13:01 wer_bpe:262] predicted:rangierllmhrt vt antworten
[NeMo I 2023-01-02 23:13:01 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:13:

Validation: 0it [00:00, ?it/s]

[NeMo I 2023-01-02 23:16:46 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:46 wer_bpe:261] reference:rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:16:46 wer_bpe:262] predicted:umiellmh ⁇ t s antworten
[NeMo I 2023-01-02 23:16:46 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:46 wer_bpe:261] reference:rangierfahrt vom gleis sieben via zulu acht ins gleis null null antworten
[NeMo I 2023-01-02 23:16:46 wer_bpe:262] predicted:umiellmh ⁇ t s antworten
[NeMo I 2023-01-02 23:16:46 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:46 wer_bpe:261] reference:spracheingabe
[NeMo I 2023-01-02 23:16:46 wer_bpe:262] predicted:umiellmh ⁇ t s antworten
[NeMo I 2023-01-02 23:16:46 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:46 wer_bpe:261] reference:rangierfahrt von quebec f ⁇ nf acht ins gleis oscar sieben drei antworten
[NeMo I 2023-01-02 23:16:46 wer_bpe:262] predicted:umiellmh ⁇ t s antworten
[NeMo I 2023-01-02 23:16:47 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:16:47 wer_b

Validation: 0it [00:00, ?it/s]

[NeMo I 2023-01-02 23:20:30 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:20:30 wer_bpe:261] reference:rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:20:30 wer_bpe:262] predicted:umsterllmanrver vs antworten
[NeMo I 2023-01-02 23:20:30 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:20:30 wer_bpe:261] reference:rangierfahrt vom gleis sieben via zulu acht ins gleis null null antworten
[NeMo I 2023-01-02 23:20:30 wer_bpe:262] predicted:umsterllmanrver vs antworten
[NeMo I 2023-01-02 23:20:31 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:20:31 wer_bpe:261] reference:spracheingabe
[NeMo I 2023-01-02 23:20:31 wer_bpe:262] predicted:umsterllmanrver vs antworten
[NeMo I 2023-01-02 23:20:31 wer_bpe:260] 
    
[NeMo I 2023-01-02 23:20:31 wer_bpe:261] reference:rangierfahrt von quebec f ⁇ nf acht ins gleis oscar sieben drei antworten
[NeMo I 2023-01-02 23:20:31 wer_bpe:262] predicted:umsterllmanrver vs antworten
[NeMo I 2023-01-02 23:20:31 wer_bpe:260] 
    
[NeMo I 2023-01-0